# Creating Flag Parameter

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')
print(incremental_flag)

0


#Creating Dimension Model

In [0]:
df_src = spark.sql('''
select Distinct(Model_ID) as Model_ID, model_category from parquet.`abfss://silver@thadiyalcardatalake.dfs.core.windows.net/carsales`
''')


In [0]:
df_src.limit(10).display()

Model_ID,model_category
Mah-M167,Mah
Che-M47,Che
Toy-M205,Toy
BMW-M249,BMW
Mer-M122,Mer
Hon-M215,Hon
Nis-M82,Nis
Toy-M206,Toy
Mar-M139,Mar
Ren-M207,Ren


### dim_model sink - initial and incremental

In [0]:
if spark.catalog.tableExists('CarProject.gold.dim_model'):
    df_sink = spark.sql('''
                    select dim_model_key, Model_ID, model_category from CarProject.gold.dim_model
                    ''')
else:
    df_sink = spark.sql('''
                    select 1 as dim_model_key, Model_ID, model_category from parquet.`abfss://silver@thadiyalcardatalake.dfs.core.windows.net/carsales`
                    where 1=0
                    ''')


In [0]:
df_sink.display()

dim_model_key,Model_ID,model_category


In [0]:
df_filter = df_src.join(df_sink, df_src.Model_ID == df_sink.Model_ID,'left').select(df_src.Model_ID, df_src.model_category, df_sink.dim_model_key)
df_filter.limit(10).display()

Model_ID,model_category,dim_model_key
Mah-M167,Mah,null
Che-M47,Che,null
Toy-M205,Toy,null
BMW-M249,BMW,null
Mer-M122,Mer,null
Hon-M215,Hon,null
Nis-M82,Nis,null
Toy-M206,Toy,null
Mar-M139,Mar,null
Ren-M207,Ren,null


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df_filter_old = df_filter.filter(col('dim_model_key').isNotNull())
df_filter_old.display()

Model_ID,model_category,dim_model_key


In [0]:
df_filter_new = df_filter.filter(col('dim_model_key').isNull()).select(df_filter.Model_ID, df_filter.model_category)
df_filter_new.limit(10).display()


Model_ID,model_category
Mah-M167,Mah
Che-M47,Che
Toy-M205,Toy
BMW-M249,BMW
Mer-M122,Mer
Hon-M215,Hon
Nis-M82,Nis
Toy-M206,Toy
Mar-M139,Mar
Ren-M207,Ren


### Creating Surrogate key

In [0]:
if (incremental_flag == '0'):
    max_value = 1
else:
    max_value_df = spark.sql("select max(dim_model_key) from carproject.gold.dim_model")
    max_value = max_value_df.collect()[0][0]+1

In [0]:
df_filter_new = df_filter_new.withColumn('dim_model_key',max_value + monotonically_increasing_id())


In [0]:
df_filter_new.limit(10).display()

Model_ID,model_category,dim_model_key
Mah-M167,Mah,1
Che-M47,Che,2
Toy-M205,Toy,3
BMW-M249,BMW,4
Mer-M122,Mer,5
Hon-M215,Hon,6
Nis-M82,Nis,7
Toy-M206,Toy,8
Mar-M139,Mar,9
Ren-M207,Ren,10


### Create Final DF - df_filter_old + df_filter_new

In [0]:
df_final = df_filter_new.union(df_filter_old)
df_final.limit(10).display()

Model_ID,model_category,dim_model_key
Mah-M167,Mah,1
Che-M47,Che,2
Toy-M205,Toy,3
BMW-M249,BMW,4
Mer-M122,Mer,5
Hon-M215,Hon,6
Nis-M82,Nis,7
Toy-M206,Toy,8
Mar-M139,Mar,9
Ren-M207,Ren,10


### SCD TYPE -1 (UPSERT)

In [0]:
from delta.tables import DeltaTable

In [0]:
# Incremental Run
if spark.catalog.tableExists('CarProject.gold.dim_model'):
    delta_tbl = DeltaTable.forPath(spark,'abfss://gold@thadiyalcardatalake.dfs.core.windows.net/dim_model')
    delta_tbl.alias('trg').merge(df_final.alias('src'), 'trg.dim_model_key = src.dim_model_key') \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()


# Initial Run
else:
    df_final.write.format('delta')\
        .mode('overwrite')\
            .option("path", 'abfss://gold@thadiyalcardatalake.dfs.core.windows.net/dim_model')\
                .saveAsTable('CarProject.gold.dim_model')    

In [0]:
%sql
select * from carproject.gold.dim_model limit 10

Model_ID,model_category,dim_model_key
Mah-M167,Mah,1
Che-M47,Che,2
Toy-M205,Toy,3
BMW-M249,BMW,4
Mer-M122,Mer,5
Hon-M215,Hon,6
Nis-M82,Nis,7
Toy-M206,Toy,8
Mar-M139,Mar,9
Ren-M207,Ren,10
